In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.utils.data as Data
import numpy as np
import pandas as pd
from tqdm import tqdm
# import optuna
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import math
import warnings
import random
from torch.backends import cudnn

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False



In [2]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
cudnn.benchmark = False
cudnn.deterministic = True

# 用30天的数据预测下一天
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = pd.read_csv('ETTh1.csv')
print(data)
data.dropna(axis=0, how='any')
data_x = data.iloc[:, 1:]


                      date    HUFL   HULL   MUFL   MULL   LUFL   LULL  \
0      2016-07-01 00:00:00   5.827  2.009  1.599  0.462  4.203  1.340   
1      2016-07-01 01:00:00   5.693  2.076  1.492  0.426  4.142  1.371   
2      2016-07-01 02:00:00   5.157  1.741  1.279  0.355  3.777  1.218   
3      2016-07-01 03:00:00   5.090  1.942  1.279  0.391  3.807  1.279   
4      2016-07-01 04:00:00   5.358  1.942  1.492  0.462  3.868  1.279   
...                    ...     ...    ...    ...    ...    ...    ...   
17415  2018-06-26 15:00:00  -1.674  3.550 -5.615  2.132  3.472  1.523   
17416  2018-06-26 16:00:00  -5.492  4.287 -9.132  2.274  3.533  1.675   
17417  2018-06-26 17:00:00   2.813  3.818 -0.817  2.097  3.716  1.523   
17418  2018-06-26 18:00:00   9.243  3.818  5.472  2.097  3.655  1.432   
17419  2018-06-26 19:00:00  10.114  3.550  6.183  1.564  3.716  1.462   

              OT  
0      30.531000  
1      27.787001  
2      27.787001  
3      25.044001  
4      21.948000  
...      

In [14]:

data_96_x = []
data_96_y = []
for i in range(0, len(data_x) - 95, 96):
    data_96_x.append(data_x[i: i+95])
    data_96_y.append(data_x.iloc[i+95, -1])
print(len(data_96_x), len(data_96_y))
x_train, x_test, y_train, y_test = train_test_split(np.array(data_96_x), np.array(data_96_y), test_size=0.2, random_state=1, shuffle=True)

print(len(x_train))
print(len(x_test))


181 181
144
37


In [4]:
class DataSet(Data.Dataset):
    def __init__(self, data_inputs, data_targets):
        self.inputs = torch.FloatTensor(data_inputs)
        self.label = torch.FloatTensor(data_targets)
    
    def __getitem__(self, index):
        return self.inputs[index], self.label[index]

    def __len__(self):
        return len(self.inputs)


In [7]:
Batch_Size = 8
dataSet = DataSet(np.array(x_train), list(y_train))
train_size = int(len(x_train) * 0.7)
test_size = len(y_train) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataSet, [train_size, test_size])
TrainDataLoader = Data.DataLoader(train_dataset, batch_size=Batch_Size, shuffle=True, drop_last=True)
TestDataLoader = Data.DataLoader(test_dataset, batch_size=Batch_Size, shuffle=True, drop_last=True)


In [33]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor):
        chunk = x.chunk(x.size(-1), dim=2)
        out = torch.Tensor([]).to(x.device)
        for i in range(len(chunk)):
            out = torch.cat((out, chunk[i] + self.pe[:chunk[i].size(0), ...]), dim=2)
        return out
    
def transformer_generate_tgt_mask(length, device):
    mask = torch.tril(torch.ones(length, length, device=device)) == 1
    mask = (
        mask.float()
        .masked_fill(mask == 0, float('-inf'))
        .masked_fill(mask == 1, float(0.0))
    )
    return mask

class Transformer(nn.Module):
    def __init__(self, n_encoder_inputs, n_decoder_inputs, Sequence_length, d_model=512, dropout=0.1, num_layers=8):
        super(Transformer, self).__init__()
        self.input_pos_embedding = torch.nn.Embedding(500, embedding_dim=d_model)
        self.target_pos_embedding = torch.nn.Embedding(500, embedding_dim=d_model)

        encoder_layer = torch.nn.TransformerEncoderLayer(d_model=d_model, nhead=num_layers, dropout=dropout, dim_feedforward=4*d_model)
        decoder_layer = torch.nn.TransformerDecoderLayer(d_model=d_model, nhead=num_layers, dropout=dropout, dim_feedforward=4*d_model)

        self.encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.decoder = torch.nn.TransformerDecoder(decoder_layer, num_layers=4)

        self.input_projection = torch.nn.Linear(n_encoder_inputs, d_model)
        self.output_projection = torch.nn.Linear(n_decoder_inputs, d_model)

        self.linear = torch.nn.Linear(d_model, 1)
        self.ziji_add_linear = torch.nn.Linear(Sequence_length, 1)

    def encode_in(self, src):
        src_start = self.input_projection(src).permute(1, 0, 2)
        in_sequence_len, batch_size = src_start.size(0), src_start.size(1)
        pos_encoder = (torch.arange(0, in_sequence_len, device=src.device).unsqueeze(0).repeat(batch_size, 1))
        pos_encoder = self.input_pos_embedding(pos_encoder).permute(1, 0, 2)
        src = src_start + pos_encoder
        src = self.encoder(src) + src_start
        return src

    def decode_out(self, tgt, memory):
        tgt_start = self.output_projection(tgt).permute(1, 0, 2)
        out_sequence_len, batch_size = tgt_start.size(0), tgt_start.size(1)
        pos_decoder = (torch.arange(0, out_sequence_len, device=tgt.device).unsqueeze(0).repeat(batch_size, 1))
        pos_decoder = self.target_pos_embedding(pos_decoder).permute(1, 0, 2)
        tgt = tgt_start + pos_decoder
        tgt_mask = transformer_generate_tgt_mask(out_sequence_len, tgt.device)
        out = self.decoder(tgt=tgt, memory=memory, tgt_mask=tgt_mask) + tgt_start
        out = out.permute(1, 0, 2)
        out = self.linear(out)
        return out
    
    def forward(self, src, target_in):
        src = self.encode_in(src)
        out = self.decode_out(tgt=target_in, memory=src)
        # torch.Size([batch, 3, 1])
        out = out.squeeze(2)
        out = self.ziji_add_linear(out)
        return out

model = Transformer(n_encoder_inputs=7, n_decoder_inputs=7, Sequence_length=95).to(device) # 3表示Sequence_length transformer输入数据 序列的长度

def test_main(model):
    val_epoch_loss = []
    with torch.no_grad():
        for index, (inputs, targets) in enumerate(TestDataLoader):
            inputs = torch.tensor(inputs).to(device)
            targets = torch.tensor(targets).to(device)
            inputs = inputs.float()
            targets = targets.float()
            tgt_in = torch.rand((Batch_Size, 95, 7))
            outputs = model(inputs, tgt_in)
            loss = criterion(outputs.float(), targets.float())
            val_epoch_loss.append(loss.item())
    return np.mean(val_epoch_loss)


In [35]:
epochs = 50
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.0001)
criterion = torch.nn.MSELoss().to(device)

val_loss = []
train_loss = []
best_test_loss = 10000000
for epoch in tqdm(range(epochs)):
    train_epoch_loss = []
    for index, (inputs, targets) in enumerate(TrainDataLoader):
        inputs = torch.tensor(inputs).to(device)
        targets = torch.tensor(targets).to(device)
        inputs = inputs.float()
        targets = targets.float()

        tgt_in = torch.rand((Batch_Size, 95, 7))
        outputs = model(inputs, tgt_in)
        loss = criterion(outputs.float(), targets.float())
        print('loss:', loss)
        loss.backward()
        optimizer.step()
        train_epoch_loss.append(loss.item())
    train_loss.append(np.mean(train_epoch_loss))
    val_epoch_loss = test_main(model)
    val_loss.append(val_epoch_loss)
    print('epoch:', epoch, "train_epoch_loss:", np.mean(train_epoch_loss), 'val_epoch_loss:', val_epoch_loss)

    if val_epoch_loss < best_test_loss:
        best_test_loss = val_epoch_loss
        best_model = model
        print('best_test_loss ---------------------', best_test_loss)
        torch.save(best_model.state_dict(), 'best_Transformer_trainModel.pth')
        

  0%|          | 0/50 [00:00<?, ?it/s]

loss: tensor(78.8761, grad_fn=<MseLossBackward0>)
loss: tensor(67.1492, grad_fn=<MseLossBackward0>)
loss: tensor(171.9894, grad_fn=<MseLossBackward0>)
loss: tensor(35.6237, grad_fn=<MseLossBackward0>)
loss: tensor(149.0495, grad_fn=<MseLossBackward0>)
loss: tensor(135.4158, grad_fn=<MseLossBackward0>)
loss: tensor(79.7751, grad_fn=<MseLossBackward0>)
loss: tensor(153.8423, grad_fn=<MseLossBackward0>)
loss: tensor(142.4051, grad_fn=<MseLossBackward0>)
loss: tensor(63.2607, grad_fn=<MseLossBackward0>)
loss: tensor(106.1435, grad_fn=<MseLossBackward0>)
loss: tensor(60.8025, grad_fn=<MseLossBackward0>)


  2%|▏         | 1/50 [00:10<08:40, 10.63s/it]

epoch: 0 train_epoch_loss: 103.69441254933675 val_epoch_loss: 87.50945129394532
best_test_loss --------------------- 87.50945129394532
loss: tensor(92.3335, grad_fn=<MseLossBackward0>)
loss: tensor(43.8874, grad_fn=<MseLossBackward0>)
loss: tensor(91.1822, grad_fn=<MseLossBackward0>)
loss: tensor(203.1676, grad_fn=<MseLossBackward0>)
loss: tensor(48.1998, grad_fn=<MseLossBackward0>)
loss: tensor(103.3121, grad_fn=<MseLossBackward0>)
loss: tensor(119.2066, grad_fn=<MseLossBackward0>)
loss: tensor(27.4668, grad_fn=<MseLossBackward0>)
loss: tensor(132.1879, grad_fn=<MseLossBackward0>)
loss: tensor(73.5120, grad_fn=<MseLossBackward0>)
loss: tensor(53.0347, grad_fn=<MseLossBackward0>)
loss: tensor(36.7185, grad_fn=<MseLossBackward0>)


  4%|▍         | 2/50 [00:21<08:46, 10.97s/it]

...

epoch: 40 train_epoch_loss: 71.68104235331218 val_epoch_loss: 68.52952194213867
loss: tensor(79.7849, grad_fn=<MseLossBackward0>)
loss: tensor(81.6701, grad_fn=<MseLossBackward0>)
loss: tensor(123.9685, grad_fn=<MseLossBackward0>)
loss: tensor(47.5489, grad_fn=<MseLossBackward0>)
loss: tensor(51.5551, grad_fn=<MseLossBackward0>)
loss: tensor(44.9825, grad_fn=<MseLossBackward0>)
loss: tensor(73.5851, grad_fn=<MseLossBackward0>)
loss: tensor(62.5572, grad_fn=<MseLossBackward0>)
loss: tensor(50.1163, grad_fn=<MseLossBackward0>)
loss: tensor(40.9254, grad_fn=<MseLossBackward0>)
loss: tensor(69.3248, grad_fn=<MseLossBackward0>)
loss: tensor(64.2954, grad_fn=<MseLossBackward0>)


 84%|████████▍ | 42/50 [08:02<01:34, 11.76s/it]

epoch: 41 train_epoch_loss: 65.85951073964436 val_epoch_loss: 67.93037185668945
loss: tensor(66.6172, grad_fn=<MseLossBackward0>)
loss: tensor(30.7825, grad_fn=<MseLossBackward0>)
loss: tensor(39.1367, grad_fn=<MseLossBackward0>)
loss: tensor(49.1660, grad_fn=<MseLossBackward0>)
loss: tensor(29.3542, grad_fn=<MseLossBackward0>)
loss: tensor(195.5952, grad_fn=<MseLossBackward0>)
loss: tensor(27.0508, grad_fn=<MseLossBackward0>)
loss: tensor(69.7397, grad_fn=<MseLossBackward0>)
loss: tensor(70.6169, grad_fn=<MseLossBackward0>)
loss: tensor(170.2443, grad_fn=<MseLossBackward0>)
loss: tensor(83.2825, grad_fn=<MseLossBackward0>)
loss: tensor(49.6539, grad_fn=<MseLossBackward0>)


 86%|████████▌ | 43/50 [08:14<01:22, 11.80s/it]

epoch: 42 train_epoch_loss: 73.43666076660156 val_epoch_loss: 70.84398498535157
loss: tensor(23.0042, grad_fn=<MseLossBackward0>)
loss: tensor(30.5661, grad_fn=<MseLossBackward0>)
loss: tensor(61.4727, grad_fn=<MseLossBackward0>)
loss: tensor(101.3197, grad_fn=<MseLossBackward0>)
loss: tensor(97.1071, grad_fn=<MseLossBackward0>)
loss: tensor(81.1108, grad_fn=<MseLossBackward0>)
loss: tensor(136.5038, grad_fn=<MseLossBackward0>)
loss: tensor(59.2030, grad_fn=<MseLossBackward0>)
loss: tensor(35.1549, grad_fn=<MseLossBackward0>)
loss: tensor(90.7103, grad_fn=<MseLossBackward0>)


In [5]:
# 画图
# loss图
fig = plt.figure(facecolor='white', figsize=(10, 7))
plt.xlabel('X')
plt.ylabel('Y')
plt.xlim(xmax=len(val_loss), xmin=0)
plt.ylim(ymax=max(max(train_loss), max(val_loss)), ymin=0)

x1 = [i for i in range(0, len(train_loss), 1)]
y1 = val_loss
x2 = [i for i in range(0, len(train_loss), 1)]
y2 = train_loss
colors1 = '#00CED4'
colors2 = '#DC143C'
area = np.pi * 4 ** 1
# 画散点图
plt.scatter(x1, y1, s=area,c=colors1, alpha=0.4, label='val_loss')
plt.scatter(x2, y2, s=area,c=colors2, alpha=0.4, label='train_loss')
plt.legend()
plt.sho